# 원티드 프리온보딩 AI/ML코스 선발과제 이창석

# 문제1) Tokenizer 생성하기

In [1]:
# 필요한 라이브러리 로드
import re # 특수문자 제거
from math import log # tf-idf 행렬곱 계산
import pandas as pd # 데이터프레임

In [2]:
class Tokenizer():
    def __init__(self):
        self.word_dict = {'oov' : 0}
        self.fit_checker = False

    def preprocessing(self, sequences):
        result = []
        
        '''
        문제 1-1. 텍스트 전처리 [START]
        '''
        
        def clean_text(seq):
            seq = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', ' ', seq)
            return seq

        for seq in sequences:
            seq = seq.lower() # 조건 1: 소문자 변환
            seq = clean_text(seq) # 조건 1: 특수문자 제거
            seq = seq.split() # 조건 2: 토큰화 white space 수행
            result.append((seq))
            
        '''
        문제 1-1. 텍스트 전처리 [END]
        '''
        
        return result # 각 문장을 토큰화한 결과: nested list

    def fit(self, sequences):
        self.fit_checker = False
        
        '''
        문제 1-2. 어휘 사전 구축 [START]
        '''
        
        # 조건 1: 위에서 만든 preprocessing 함수를 이용하여 각 문장 토큰화
        preprocessed_seqs = self.preprocessing(sequences)
        
        pre_result = []
        for preprocessed_seq in preprocessed_seqs:
            pre_result += preprocessed_seq # 전처리된 각 문장 별 단어를 하나의 리스트로 합침

        pre_result = set(pre_result) # set을 이용하여 문장 별 중복된 단어 합침

        for num, word in enumerate(pre_result,1): # oov가 어휘 사전에 이미 value=0으로 존재하므로 '1'부터 시작
            self.word_dict[word] = num # 조건 2: dict을 이용하여 어휘 사전 생성

            
        '''
        문제 1-2. 어휘 사전 구축 [END]
        '''
        
        self.fit_checker = True

    def transform(self, sequences):
        result = []
        tokens = self.preprocessing(sequences)

        if self.fit_checker:
            
            '''
            문제 1-3. 어휘 사전을 활용하여 입력 문장을 정수 인뎅식하는 함수 [START]
            '''

            for token in tokens:
                token2index = []
                for word in token:
                    try:
                        token2index.append((self.word_dict[word]))
                    except KeyError: # 조건 1: 어휘 사전에 없는 단어는 'oov'의 index로 변환
                        token2index.append((self.word_dict['oov']))
                result.append(token2index)
                
            '''
            문제 1-3. 어휘 사전을 활용하여 입력 문장을 정수 인뎅식하는 함수 [END]
            '''

            return result # 각 문장의 정수 인덱싱: nested list

        else:
            raise Exception("Tokenizer instance is not fitted yet.")

    def fit_transform(self, sequences):
        self.fit(sequences)
        result = self.transform(sequences)
        return result

In [3]:
# 예시 문장
sequences = ['I go to school.', 'I LIKE pizza!']
oov_sequences = ['I go to school.', 'I LIKE pizza!', 'i HaTe You!~']

tokenizer_seqs = Tokenizer()

# 1-1) preprocessing()
print('1-1) preprocessing : ', tokenizer_seqs.preprocessing(sequences))
print('\n')

# 1-2) fit()
tokenizer_seqs.fit(sequences)
print('1-2) word_dict : ', tokenizer_seqs.word_dict)
print('\n')

# 1-3) transform()
print('1-3) transform : ', tokenizer_seqs.transform(sequences))
print('1-3) oov_transform : ', tokenizer_seqs.transform(oov_sequences))
print('\n')

# 1-4) fit_transform()
print('1-4) fit_transform : ', tokenizer_seqs.fit_transform(sequences))

1-1) preprocessing :  [['i', 'go', 'to', 'school'], ['i', 'like', 'pizza']]


1-2) word_dict :  {'oov': 0, 'pizza': 1, 'i': 2, 'go': 3, 'like': 4, 'to': 5, 'school': 6}


1-3) transform :  [[2, 3, 5, 6], [2, 4, 1]]
1-3) oov_transform :  [[2, 3, 5, 6], [2, 4, 1], [2, 0, 0]]


1-4) fit_transform :  [[2, 3, 5, 6], [2, 4, 1]]


---

# 문제2) TfidfVectorizer 생성하기

In [4]:
class TfidfVectorizer:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.fit_checker = False

    def fit(self, sequences):
        tokenized = self.tokenizer.fit_transform(sequences)
        
        '''
        문제 2-1. 입력 문장들을 이용해 IDF 행렬을 만드는 함수 [START]
        '''
        
        vocab = list(self.tokenizer.word_dict.keys()) # 문제 1에서 만든 word dict 사용함
        n = len(sequences)
        
        merged_seqs = []
        for i in tokenized: # 조건 3: 문제 1에서 만든 Tokenizer 사용
            num2word = []
            for id in i:
                word_dict = vocab[id]
                num2word.append(word_dict)
            num2word = ' '.join(num2word)
            merged_seqs.append(num2word)
        
        def idf(t): # 조건 2: idf 계산하는 함수: n은 입력된 전체 문장 개수, df는 단어 t가 포함된 문자 d의 개수
            df = 0
            for seq in merged_seqs:
                df += t in seq
            return log(n/(df+1))        

        idf_matrix = []
        for j in range(len(vocab)):
            t = vocab[j]
            idf_matrix.append(idf(t))

        self.idf_matrix = idf_matrix # 조건 1: IDF 행렬: list
        
        '''
        문제 2-1. 입력 문장들을 이용해 IDF 행렬을 만드는 함수 [END]
        '''
        
        self.fit_checker = True

    def transform(self, sequences):
        if self.fit_checker:
            tokenized = self.tokenizer.transform(sequences)
        
            '''
            문제 2-2. 입력 문장들을 이용해 TF-IDF 행렬을 만드는 함수 [START]
            '''
            
            vocab = list(self.tokenizer.word_dict.keys())
            n = len(sequences)
            
            merged_seqs = []
            for i in tokenized: # 문제 1에서 만든 Tokenizer 사용
                num2word = []
                for id in i:
                    word_dict = vocab[id]
                    num2word.append(word_dict)
                num2word = ' '.join(num2word)
                merged_seqs.append(num2word)
                
            def tf(t, s): # tf 계산하는 함수: t는 특정 단어가 등장 횟수, s는 특정 문장
                cnt = 0
                ss = s.split( )
                for s in ss:
                    if t == s:
                        cnt += 1
                return cnt
            
            tf_matrix = []
            for i in range(n):
                tf_matrix.append([])
                s = merged_seqs[i]
                for j in range(len(vocab)):
                    t = vocab[j]
                    tf_matrix[-1].append(tf(t,s))
                    
            self.tf_matrix = tf_matrix # 조건 1: TF행렬 생성

            tfidf_matrix = []
            for idx in range(n): # 조건 2: TF행렬과 문제 2-1에서 만든 IDF행렬을 곱하여 TF-IDF행렬 생성
                product = [x*y for x,y in zip(tf_matrix[idx],self.idf_matrix)]
                tfidf_matrix.append(product)
            self.tfidf_matrix = tfidf_matrix
            
            '''
            문제 2-2. 입력 문장들을 이용해 TF-IDF 행렬을 만드는 함수 [END]
            '''
            
            return self.tfidf_matrix # TF-IDF행렬: nested list

        else:
            raise Exception("TfidfVectorizer instance is not fitted yet.")

    def fit_transform(self, sequences):
        self.fit(sequences)
        return self.transform(sequences)

In [5]:
# 예시 문장
sequences = ['I go to school.', 'I LIKE pizza!']
# oov_sequences = ['I go to school.', 'I LIKE pizza!', 'i HaTe You!~']

tokenizer_seqs = Tokenizer()

Tfidf_seqs = TfidfVectorizer(tokenizer_seqs)

Tfidf_seqs.fit(sequences)
Tfidf_seqs.transform(sequences)

print('word dict : ', tokenizer_seqs.word_dict)
print('\n')

# 2-1) fit()
print('2-1) IDF matrix : ', Tfidf_seqs.idf_matrix)
print('\n')

# 2-2) transform()
print('2-2)')
print('\n')
print('TF matrix : ')
print(Tfidf_seqs.tf_matrix)
print('\n')
print('TF-IDF matrix : ')
print(Tfidf_seqs.tfidf_matrix)
print('\n')

word dict :  {'oov': 0, 'pizza': 1, 'i': 2, 'go': 3, 'like': 4, 'to': 5, 'school': 6}


2-1) IDF matrix :  [0.6931471805599453, 0.0, -0.40546510810816444, 0.0, 0.0, 0.0, 0.0]


2-2)


TF matrix : 
[[0, 0, 1, 1, 0, 1, 1], [0, 1, 1, 0, 1, 0, 0]]


TF-IDF matrix : 
[[0.0, 0.0, -0.40546510810816444, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.40546510810816444, 0.0, 0.0, 0.0, 0.0]]




---

![IMG_4A8190CE40A9-1](https://user-images.githubusercontent.com/67947808/153157566-0e7a354f-386b-41f2-9c47-5523ef2816cf.jpeg)